# Extracting desired business reviews
In the last notebook we achieved two things. Firstly, we identified the business categories (Italian/Pizza) that our campaign was going to target. Secondly, we identified business IDs associated with those categories. Knowing the business IDs allows us to select just the reviews of interest from the reviews file. Before you go rushing into reading the review data into one big dataframe to then filter just those IDs of interest, as a data scientist you will have first got an idea of the size of the file. The review file is pretty big (nearly six million lines). Unless you have a very good computer with lots of RAM, or you particularly want to fire up your system/memory monitor and watch the free memory fall further and further until your computer seizes, you should be thinking in terms of how to read in only the lines of interest. This way, you only consume the minimum RAM necessary. Of course, in general, even then you should be making some sort of calculation as to whether even this would be within the capabilities of your hardware. In this case, it should be doable for most modernish computers. The task of this notebook is to parse the review data file and save the (much smaller) subset of interest.

In [8]:
# import the required libraries here
import pandas as pd
from matplotlib import pyplot as plt

## List of required business IDs
First we need to read in the list of business IDs we obtained previously.

In [9]:
# task: read in our previously created file that contains the business IDs of interest
# one line
businesses = pd.read_csv('campaign_list')

It's a good idea to have a quick check first.

In [10]:
businesses.head()

,business_id,review_count,stars,state
0,ZkzutF0P_u0C0yTulwaHkA,31,4.0,PA
1,OWkS1FXNJbozn-qPg3LWxg,11,4.5,NV
2,6GHwgKNlvfIMUpFaxgBjUA,9,4.0,QC
3,-C0AlwLuXpcP609madJZQQ,7,3.5,ON
4,39lLJK_rrYY2NYomSsQdUA,11,2.5,OH


We want to grab just the business_id column as a list. It's worth also then double checking it matches the business_id column above.

In [11]:
business_ids = businesses['business_id'].values
print(business_ids[:5])

['ZkzutF0P_u0C0yTulwaHkA' 'OWkS1FXNJbozn-qPg3LWxg'
 '6GHwgKNlvfIMUpFaxgBjUA' '-C0AlwLuXpcP609madJZQQ'
 '39lLJK_rrYY2NYomSsQdUA']


## Filtering a large data file in chunks
The review data file comprises some nearly 6 million lines. We don't want all of those lines. Our general approach will be to read through the file and check the business_id in each row. If this ID is contained in our list of wanted IDs, we will keep that row. It will be fastest, but most expensive for memory, to read the entire file from disk in one go and then filter the rows in memory. At the other extreme it will be most memory efficient to read in one record at a time, check whether we want it or not (and discard if not), but slowest. We will strike a balance where we choose a sufficiently large chunksize that still fits easily in memory but whereby we don't need to read in too many chunks.

In [12]:
# For convenience here, again we are accessing the data in the working directory that contains our notebooks.
# Normal best practise is to keep your data separate, but this keeps things simple here.
# If you're comfortable specifying a filepath to files outside of this working directory, then feel free to do so.
# task: create a reader object for the review json file
# Hint: use lines=True as before but add the chunksize=100000 parameter
# one line of code here
review_reader = pd.read_json('yelp_academic_dataset_review.json',lines=True,chunksize=100000)

We use the time magic for the next cell, out of interest, to time how long it takes. If you have the time and are curious, you're welcome to play around with the chunksize and monitor your system memory use and the time this cell takes, but _caveat emptor_! I suggest you save your notebook before running this cell, just in case.

In [13]:
%%time
# task: process the file one chunk at a time,
# filter that chunk for rows with a business_id in business_ids
# You can either do this within in a loop, having initialized an empty list,
# or using a more pythonic list comprehension
reviews = [review.loc[review['business_id'].apply(lambda x: x in business_ids)] for review in review_reader]
# (this took some 24 minutes on my old i7)

Wall time: 35min 41s


All being well, you now have the desired reviews read in. But we don't yet have them in a convenient DataFrame:

In [14]:
type(reviews)

list

In [15]:
len(reviews)

81

Convert your reviews into a DataFrame now. _Hint: you may find pandas concat method useful here._

In [16]:
# task convert your reviews into a DataFrame
# one line of code here
reviews = pd.concat(reviews)

In [17]:
reviews.shape

(712958, 9)

In [18]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
8,JA-xnyHytKiOIHl_ztnK9Q,P6apihD4ASf1vpPxHODxAQ,e_BiI4ej1CW1F0EyVLr-FQ,5,0,0,0,Most delicious authentic Italian I've had in t...,2015-02-16 06:48:47
25,mM8i91yWP1QbImEvz5ds0w,TZQSUDDcA4ek5gBd6BzcjA,qUWqjjjfpB2-4P3He5rsKw,4,0,0,0,"In the heart of Chinatown, I discovered it enr...",2017-06-06 19:04:10
27,09qxjFi4abaW66JeSLazuQ,mbdtyUUzZZx5ld1Qc4iGtQ,wkzWdo1mBqbzR2KPoXtWZw,4,1,0,0,Was a Chicago style deep dish. Homemade type ...,2010-08-21 01:19:17
28,K-wdPGHbErfxbKK6PetrmA,FCoXqPxuI7GRBYyzJIwVxw,r4HkiC71nB9k8Fe8zIORFQ,5,1,0,1,First time eating there and everything was so ...,2015-12-05 02:16:06
44,nAm92_-WUQ084x9Uz9iFmA,iFEM16O5BeKvf_AHs6NzUQ,9Eghhu_LzEJgDKNgisf3rg,3,0,0,0,Was super excited to bring my boyfriend to thi...,2016-08-25 16:25:12


## Save the output
Having done all this hard work filtering our review file, and documenting the process in this notebook, we need to save the output. Go ahead and save the reviews DataFrame to a csv called 'reviews_filtered.csv'.

In [19]:
# task: save the DataFrame to the specified file now.
# don't forget to use index=False
reviews.to_csv('filtered_reviews',index=False)

If you inspect the file listing for the original json file and our new csv file, you should see we've gone from around 4.4 GB down to 325 MB. This is much more manageable!

# Summary
Great work! So far in our project you've seen how to break a problem down and identify relevant data, use data to gain insight and make a decision, and then leverage that knowledge to extract a desired subset of data from an otherwise unmanageable larger file. In the next notebook we'll dive into this data.